In [3]:
import numpy as np
import pandas as pd
import time

In [4]:
df = pd.read_table('seeds_dataset.txt', sep='\s+',header=None)

In [5]:
df.head()

,0,1,2,3,4,5,6,7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [6]:
data = np.array(df)
X = data[:,0:7]
label_true = data[:,7]

In [7]:
X.shape

(210, 7)

In [8]:
rng = np.random.RandomState()
i = rng.permutation(X.shape[0])[:3]
X[i]

array([[11.48  , 13.05  ,  0.8473,  5.18  ,  2.758 ,  5.876 ,  5.002 ],
       [18.17  , 16.26  ,  0.8637,  6.271 ,  3.512 ,  2.853 ,  6.273 ],
       [12.01  , 13.52  ,  0.8249,  5.405 ,  2.776 ,  6.992 ,  5.27  ]])

In [56]:
class kmeans():
    def __init__(self,nclusters,random = False):
        self.nclusters = nclusters
        self.random = random
    def fit(self,data):
        t1 = time.time()
        if self.random:
            rng = np.random.RandomState()
        else:
            rng = np.random.RandomState(12)
        i = rng.permutation(data.shape[0])[:self.nclusters]
        centers = data[i]
#         centers = np.random.random((self.nclusters,data.shape[1]))
        C = np.zeros([self.nclusters,data.shape[1]])
    # iterates until converges
        iterations = 0
        while True:
            iterations+=1
            # assign
            labels = []
            for i in range(data.shape[0]):
                dist = []
                for j in range(self.nclusters):
                    dist.append(np.linalg.norm(data[i]-centers[j]))
                labels.append(np.argmin(dist)+1)# Add 1 since the true label start from '1'
            print(labels)
            # get new centers
            for i in range(self.nclusters):
                c = np.mean(data[np.array(labels)==(i+1)],axis=0)# center for this cluster
                if not np.isnan(c).any():
                    C[i] = c.copy()
                else:
                    C[i] = np.zeros(data.shape[1])
            if (C == centers).all():
                break
            centers = C.copy()
        t2 = time.time()
        self.time = t2 - t1
        self.iterations = iterations
        self.centers = centers
        return centers
    def predict(self,data):
        labels = []
        for i in range(data.shape[0]):
            dist = []
            for j in range(self.nclusters):
                dist.append(np.linalg.norm(data[i]-self.centers[j]))
            labels.append(np.argmin(dist)+1)# Add 1 since the true label start from '1'
        return np.array(labels)

In [54]:
class accelerated_kmeans():
    def __init__(self,nclusters,random = False):
        self.nclusters = nclusters
        self.random = random
    def fit(self,data):
        t1 = time.time()
        if self.random:
            rng = np.random.RandomState()
        else:
            rng = np.random.RandomState(12)
        i = rng.permutation(data.shape[0])[:self.nclusters]
        centers = data[i]
        labels = []
        lxc = []
        ux = []
        for i in range(data.shape[0]):
            label = 0
            lc = [0 for i in range(self.nclusters)]
            for j in range(self.nclusters):
                if j==0:
                    label = 1
                    dxc = np.linalg.norm(data[i]-centers[j])
                    lc[j] = dxc
                else:
                    if np.linalg.norm(centers[label-1]-centers[j]) < 2*np.linalg.norm(data[i]-centers[label-1]):
                        dxc_new = np.linalg.norm(data[i]-centers[j])
                        lc[j] = dxc_new
                        if dxc > dxc_new:
                            dxc = dxc_new
                            label = j + 1
            labels.append(label)
            lxc.append(lc)
            ux.append(dxc)
        r = [True for i in range(data.shape[0])]
        iterations = 0
        # Repeat 1-7 until convergence
        while True:
                print(centers)
                iterations+=1
                #1.
                sc = []
                dcc = np.zeros((self.nclusters,self.nclusters))
                for i in range(self.nclusters):
                    for j in range(i+1,self.nclusters):
                        dcc[i][j] = np.linalg.norm(centers[i]-centers[j])
                        dcc[j][i] = dcc[i][j]
                for i in range(self.nclusters):
                    d = []
                    for j in range(self.nclusters):
                        if i != j:
                            d.append(np.linalg.norm(centers[i]-centers[j]))
                    sc.append(0.5*min(d))
                for i in range(data.shape[0]):
                    #2.
                    if ux[i] <= sc[labels[i]-1]:
                        pass
                    #3.
                    else:
                        for j in range(self.nclusters):
                            #3.
                            if (j != labels[i]-1) and (ux[i] > lxc[i][j]) and (ux[i] > 0.5*dcc[labels[i]-1][j]):
                                #3a.
                                if r[i]:
                                    dx_cx = np.linalg.norm(data[i]-centers[labels[i]-1])
                                    ux[i] = dx_cx
                                    r[i] = False
                                else:
                                    dx_cx = ux[i]
                                #3b.
                                if (dx_cx > lxc[i][j]) or (dx_cx > 0.5*dcc[labels[i]-1][j]):
                                    dxc = np.linalg.norm(data[i]-centers[j])
                                    lxc[i][j] = dxc
                                    if dxc < dx_cx:
                                        labels[i] = j+1
                                        ux[i] = dxc
                #4.
                mc = []
                for j in range(self.nclusters):
                    mc.append(np.mean(data[np.array(labels)==(j+1)],axis=0))
                mc = np.array(mc)#to np array 
                for i in range(data.shape[0]):
                    #5.
                    for j in range(self.nclusters):
                        lxc[i][j] = max(0,lxc[i][j]-np.linalg.norm(centers[j]-mc[j]))
                    #6.
                    ux[i] = ux[i] + np.linalg.norm(mc[labels[i]-1]-centers[labels[i]-1])
                    r[i] = True
                if (mc == centers).all():
                    break
                #7
                centers = mc.copy()
        t2 = time.time()
        self.time = t2 - t1
        self.iterations = iterations
        self.centers = centers
        return centers
    def predict(self,data):
        labels = []
        for i in range(data.shape[0]):
            dist = []
            for j in range(self.nclusters):
                dist.append(np.linalg.norm(data[i]-self.centers[j]))
            labels.append(np.argmin(dist)+1)# Add 1 since the true label start from '1'
        return np.array(labels)

In [55]:
k = accelerated_kmeans(3)
k.fit(X)
k.predict(X)

[[14.03   14.16    0.8796  5.438   3.201   1.717   5.001 ]
 [17.55   15.66    0.8991  5.791   3.69    5.366   5.661 ]
 [12.19   13.2     0.8783  5.137   2.981   3.631   4.87  ]]
[[14.64087719 14.45333333  0.87985263  5.55961404  3.28196491  2.34765263
   5.16091228]
 [18.34971429 16.13842857  0.88402714  6.15524286  3.68014286  3.7234
   6.00985714]
 [12.03578313 13.30024096  0.85393012  5.2316506   2.88704819  4.60949398
   5.07027711]]
[[14.55470588 14.41191176  0.87948824  5.54992647  3.26808824  2.56154706
   5.15895588]
 [18.56923077 16.23507692  0.88438615  6.18372308  3.70556923  3.63626154
   6.04723077]
 [11.96441558 13.27480519  0.8522      5.22928571  2.87292208  4.75974026
   5.08851948]]
[[14.62366197 14.44929577  0.87906197  5.56160563  3.27442254  2.62684789
   5.18632394]
 [18.68451613 16.28048387  0.88511129  6.20101613  3.71948387  3.61348387
   6.05887097]
 [11.96441558 13.27480519  0.8522      5.22928571  2.87292208  4.75974026
   5.08851948]]
[[14.64847222 14.46041

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 3, 1, 1,
       1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1,
       1, 1, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2,
       1, 1, 1, 1, 2, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3])

In [57]:
a = kmeans(3)
a.fit(X)
a.predict(X)
a.centers

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 3, 1, 1, 3, 3, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1, 3, 1, 1, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 1, 1, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

array([[14.64847222, 14.46041667,  0.87916667,  5.56377778,  3.27790278,
         2.64893333,  5.19231944],
       [18.72180328, 16.29737705,  0.88508689,  6.20893443,  3.72267213,
         3.60359016,  6.06609836],
       [11.96441558, 13.27480519,  0.8522    ,  5.22928571,  2.87292208,
         4.75974026,  5.08851948]])

In [29]:
a.time

0.03818321228027344

In [33]:
class GMM():
    def __init__(self,nclusters,tolerance=1e-6,random = False):
        self.nclusters = nclusters
        self.tolerance = tolerance
        self.random = random
    def gaussian(self, x, mean, cov):
        left = 1/(pow((2*np.pi),0.5*x.shape[0])*pow(np.linalg.det(cov),0.5))
        right = np.exp(-0.5 *(x-mean).dot(np.linalg.inv(cov)).dot(x-mean))
        return  left*right
    def initialization(self,data):
        clu = accelerated_kmeans(3,self.random) # use k mean to get the initial weights, means and covariances
        means = clu.fit(data)
        labels = np.array(clu.predict(data))
        print(labels)
        weights = []
        covariances = []
        for j in range(self.nclusters):
            X = data[labels==j+1]
            weights.append(X.shape[0]/data.shape[0])
            u = 0
            for x in data:
                mul = x - means[j]
                u = u + np.matmul(mul.reshape(data.shape[1],1),mul.reshape(1,data.shape[1]))
            print(X.shape[0])
            covariance = u/X.shape[0]
            covariances.append(covariance)
        return means, weights, covariances
    def fit(self,data):
        t1 = time.time()
        means, weights, covariances = self.initialization(data)
        print(means)
        #initial log likehood
        llh = 0
        for i in range(data.shape[0]):
            l = 0
            for j in range(self.nclusters):
                l += weights[j]*self.gaussian(data[i],means[j],covariances[j])
            llh += np.log(l)
        iterations = 0
        while True:
#             print(llh)
#             print(1)
#             print(means)
            iterations+=1
            # E-step:calculate gamma
            gamma = []
            for i in range(data.shape[0]):
                gk = []
                s = 0
                for j in range(self.nclusters):
                    s += weights[j] * self.gaussian(data[i],means[j],covariances[j])
                for j in range(self.nclusters):
                    value = weights[j] * self.gaussian(data[i],means[j],covariances[j])
                    gk.append(value/s)
                gamma.append(gk)
            # M-step
            # update mean-k
            for j in range(self.nclusters):
                u = 0
                for i in range(data.shape[0]):
                    u += gamma[i][j] * data[i]
                means[j] = u / np.sum(gamma,axis=0)[j]
            #update covariances
            for j in range(self.nclusters):
                e = 0
                for i in range(data.shape[0]):
                    mul = data[i] - means[j]
                    e = e +  gamma[i][j]*np.matmul(mul.reshape(data.shape[1],1),mul.reshape(1,data.shape[1]))
                    covariances[j] = e/np.sum(gamma,axis=0)[j]
            #update weights
                weights[j] = np.sum(gamma,axis = 0)[j] / data.shape[0]
            #update log likehood
            new_llh = 0
            for i in range(data.shape[0]):
                l = 0
                for j in range(self.nclusters):
                    l += weights[j]*self.gaussian(data[i],means[j],covariances[j])
                new_llh += np.log(l)
                
            
            if abs(new_llh - llh)<self.tolerance:
                    break
            llh = new_llh
        t2 = time.time()
        self.time = t2 - t1
        self.iterations = iterations
        self.means = means
        self.weights = weights
        self.covariances = covariances
        return means,weights,covariances
    
    def predict(self,data):
        gamma = []
        for i in range(data.shape[0]):
            gk = []
            s = 0
            for j in range(self.nclusters):
                s += self.weights[j] * self.gaussian(data[i],self.means[j],self.covariances[j])
            for j in range(self.nclusters):
                value = self.weights[j] * self.gaussian(data[i],self.means[j],self.covariances[j])
                gk.append(value/s)
            gamma.append(gk)
        labels = []
        for g in gamma:
            labels.append(np.argmax(g)+1)
        return np.array(labels)

In [34]:
g1 = GMM(3)
g1.fit(X)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1
 3 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 1 1 1 1 1 2 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 1 3 1 3 3 3 3 3 3 3 1 1 1 1 3 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2]
72
77
61
[[14.64847222 14.46041667  0.87916667  5.56377778  3.27790278  2.64893333
   5.19231944]
 [11.96441558 13.27480519  0.8522      5.22928571  2.87292208  4.75974026
   5.08851948]
 [18.72180328 16.29737705  0.88508689  6.20893443  3.72267213  3.60359016
   6.06609836]]
312.32441765027056
1
[[14.64847222 14.46041667  0.87916667  5.56377778  3.27790278  2.64893333
   5.19231944]
 [11.96441558 13.27480519  0.8522      5.22928571  2.87292208  4.75974026
   5.08851948]
 [18.72180328 16.29737705  0.88508689  6.20893443  3.72267213  3.60359016
   6.06609836]]
730.847

1209.1924975320833
1
[[14.99977574 14.61345623  0.88197557  5.63267785  3.32205447  2.89878023
   5.25276507]
 [11.99777766 13.28815667  0.85299564  5.23178304  2.87833474  4.43811816
   5.07304689]
 [18.4050157  16.15927874  0.88387098  6.14016649  3.69196412  3.50955226
   5.99235444]]
1209.1924991474339
1
[[14.99973059 14.61343435  0.88197561  5.6326688   3.32204955  2.89876907
   5.25275255]
 [11.99777625 13.28815616  0.85299561  5.23178295  2.87833448  4.43811897
   5.07304688]
 [18.40498603 16.15926648  0.8838709   6.1401642   3.6919609   3.50955124
   5.99235083]]
1209.1925001793384
1
[[14.99969453 14.61341686  0.88197564  5.63266157  3.32204562  2.89876014
   5.25274252]
 [11.99777516 13.28815577  0.85299558  5.23178287  2.87833428  4.4381196
   5.07304687]
 [18.40496212 16.1592566   0.88387084  6.14016235  3.6919583   3.50955037
   5.99234793]]


(array([[14.99966572, 14.61340288,  0.88197567,  5.63265579,  3.32204249,
          2.89875299,  5.2527345 ],
        [11.9977743 , 13.28815546,  0.85299556,  5.23178282,  2.87833412,
          4.4381201 ,  5.07304687],
        [18.40494289, 16.15924866,  0.88387079,  6.14016085,  3.69195621,
          3.50954963,  5.99234559]]),
 [0.2942229077103933, 0.398851457369189, 0.3069256349204178],
 [array([[ 1.02419350e+00,  4.96667425e-01, -3.28036296e-04,
           2.00243868e-01,  1.08665957e-01,  1.52000947e-01,
           2.63890376e-01],
         [ 4.96667425e-01,  2.56426955e-01, -2.05126588e-03,
           1.13533110e-01,  4.34652201e-02,  8.25291528e-02,
           1.46493574e-01],
         [-3.28036296e-04, -2.05126588e-03,  2.31169676e-04,
          -2.08837417e-03,  1.09745356e-03, -1.10182634e-03,
          -2.37422521e-03],
         [ 2.00243868e-01,  1.13533110e-01, -2.08837417e-03,
           5.92424828e-02,  1.13707683e-02,  5.05575627e-02,
           7.39805693e-02],
      

In [35]:
g1.time

3.6772141456604004

In [58]:
def Silhouette_Coefficient_One(data,labels,index):
    k = np.unique(labels).size
    a = 0
    b = 0
    label = labels[index]
    for i in range(k):
        d = 0
        X = data[np.array(labels) == i + 1]
        for x in X:
            d += np.linalg.norm(x - data[index])
        if label == i+1:
            d = d/(X.shape[0]-1)
            a = d
        else:
            d = d/X.shape[0]
            if b == 0:
                b = d
            elif d < b:
                b = d
    return (b-a)/max(a,b)

def Silhouette_Coefficient(data,labels):
    Silhouette_Coefficients = []
    for i in range(data.shape[0]):
        Silhouette_Coefficients.append(Silhouette_Coefficient_One(data,labels,i))
    return np.mean(Silhouette_Coefficients)

In [60]:
Silhouette_Coefficient(X,a.predict(X))

0.47193373191268945

In [64]:
def Rand_Index(label_predict,label_true):
    a = b = c = d =0
    for i in range(len(label_predict)):
        for j in range(i+1,len(label_predict)):
            if label_predict[i] == label_predict[j] and label_true[i] == label_true[j]:
                a += 1
            if label_predict[i] != label_predict[j] and label_true[i] != label_true[j]:
                b += 1
            if label_predict[i] != label_predict[j] and label_true[i] == label_true[j]:
                c += 1
            if label_predict[i] == label_predict[j] and label_true[i] != label_true[j]:
                d += 1
    return (a+b)/((a+b+c+d))

In [92]:
from sklearn.mixture import GaussianMixture
g = GaussianMixture(3)

In [94]:
g.fit_predict(X)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2,
       2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 1, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 2, 1, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       2, 2, 2, 2, 0, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
g.means_

In [ ]:
a = np.array([[1,3],[5,4]])
np.argmax(a,axis = 1)

In [ ]:
sum(a)

In [ ]:
np.log(2)

In [ ]:
a = [[1,3],[2,100]]
np.sum(a,axis=0)

In [ ]:
np.sum(a,axis=0)

In [61]:
from sklearn.metrics import silhouette_score, rand_score

In [62]:
silhouette_score(X,a.predict(X))

0.4719337319126887

In [72]:
Silhouette_Coefficient(X,g1.predict(X))

0.4488813612221242

In [65]:
Rand_Index(k.predict(X),label_true)

0.8743677375256322

In [66]:
rand_score(k.predict(X),label_true)

0.8743677375256322

In [52]:
from sklearn.cluster import KMeans
z = KMeans(3)
z.fit(X)
Rand_Index(k.predict(X),label_true)

0.8713602187286398

In [60]:
def variance(scores):
    s = 0
    mean = np.mean(scores)
    for score in scores:
        s += (score - mean)**2
    return s/len(scores)
def sensitivity(model,data,times):
    se = []
    ri = []
    for i in range(times):
        model.fit(data)
        predict = model.predict(data)
        se.append(Silhouette_Coefficient(data,model.predict(data),3))
        ri.append(Rand_Index(predict,label_true))
    se_variance=variance(se)
    ri_variance = variance(ri)
    return se_variance,ri_variance

In [63]:
evaluate(kmean(3),X,10)

(3.0814879110195774e-33, 0.0)

In [59]:
np.unique([1,3,1])

2